##IMPORTS

In [1]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import scipy.io.wavfile as wav
import tensorflow as tf
import cv2 
import numpy as np
from matplotlib import pyplot as plt
import numpy as np
from math import exp
from google.colab.patches import cv2_imshow as show
import re
import os
import shutil
import librosa

##FACE FEATURES

###LBP

In [6]:
#LBP

def get_pixel(img, center, x, y):
	new_value = 0
	try:
		if img[x][y] >= center:
			new_value = 1	
	except:
		pass
	return new_value

def lbp_calculated_pixel(img, x, y):
	center = img[x][y]
	value = []
	
	value.append(get_pixel(img, center, x-1, y-1)) #top_left	
	value.append(get_pixel(img, center, x-1, y)) #top	
	value.append(get_pixel(img, center, x-1, y + 1)) #top_right	
	value.append(get_pixel(img, center, x, y + 1)) #right	
	value.append(get_pixel(img, center, x + 1, y + 1)) #bottom_right	
	value.append(get_pixel(img, center, x + 1, y)) #bottom	
	value.append(get_pixel(img, center, x + 1, y-1)) #bottom_left	
	value.append(get_pixel(img, center, x, y-1)) #left
	
	return value


###LTP

In [7]:
#LTP

def get_pixel(img, center, x, y):
	new_value = 0
	threshold = 7  #given in paper 
	try:
		if img[x][y] >= center + threshold:
			new_value = 1
		elif img[x][y] <= center - threshold:
			new_value = -1
	except:
		pass
	return new_value

def ltp_calculated_pixel(img, x, y):
  center = img[x][y]
  value_pos = []      
  value_neg = []

  value_pos.append(get_pixel(img, center, x-1, y-1)) #top_left	
  value_pos.append(get_pixel(img, center, x-1, y)) #top	
  value_pos.append(get_pixel(img, center, x-1, y + 1)) #top_right	
  value_pos.append(get_pixel(img, center, x, y + 1)) #right	
  value_pos.append(get_pixel(img, center, x + 1, y + 1)) #bottom_right	
  value_pos.append(get_pixel(img, center, x + 1, y)) #bottom	
  value_pos.append(get_pixel(img, center, x + 1, y-1)) #bottom_left	
  value_pos.append(get_pixel(img, center, x, y-1)) #left
  
  value_neg = value_pos.copy()
  value_neg = list(map(lambda x: abs(x) if x<=0 else 0, value_pos))
  # print(value_neg)
  value_pos = list(map(lambda x: x if x>=0 else 0, value_pos))
  return value_pos, value_neg

# def ltpbinary(value_pos, value_neg):
#   power_val = [1, 2, 4, 8, 16, 32, 64, 128]
#   sum1 = 0
#   sum2 = 0
  
#   for i in range(len(value_pos)):
#     sum1 += value_pos[i] * power_val[i]
#     sum2 += value_neg[i] * power_val[i]
#   return sum1, sum2

###Orthogonal Combination of LBP and LTP


In [8]:
def sigmoid(x, slope, threshold):
    return 1/(1+exp(slope*(threshold-x)))

In [9]:
def binary(value):	
	power_val = [1, 2, 4, 8, 16, 32, 64, 128]
	sum = 0
	
	for i in range(len(value)):
		sum = sum + (value[i] * power_val[i])
	return sum

In [10]:
def corner_neighbour(value):
  neighbour = []
  corner = []
  for i in range(0,8):
        if i%2 != 0:
            neighbour.append(value[i])
            corner.append(0)
        
        elif i%2 == 0:
            neighbour.append(0)
            corner.append(value[i])
  return corner, neighbour

In [11]:
#this will be for entire image at once - donot change height and width values 
#orthogonal

def ortho (img):
  height = img.shape[0]
  width = img.shape[1]
  orthoimg =  np.empty((height, width))
  slope = 0.5      #random
  threshold = 7    #random

  # print(img)

  img_ltp_pos = np.zeros((height, width), np.uint8)
  img_ltp_neg = np.zeros((height, width), np.uint8)
  img_lbp = np.zeros((height, width), np.uint8)

  for i in range(0, height):
      for j in range(0, width):
          value_pos, value_neg = ltp_calculated_pixel(img, i, j)
          # img_ltp_pos[i, j], img_ltp_neg[i, j] = ltpbinary(value_pos, value_neg)
          # print(img_ltp_pos)
          # print(img_ltp_neg)

          value_lbp = lbp_calculated_pixel(img, i, j)
          # img_lbp[i, j] = lbpbinary(value_lbp)
          # print(img_lbp)

          #Orthogonal LBP and LTP
          lbp_corner , lbp_neigbour = corner_neighbour(value_lbp)
          value_pos_corner, value_pos_neighbour = corner_neighbour(value_pos)
          value_neg_corner, value_neg_neighbour = corner_neighbour(value_neg)
          # print(lbp_corner)
          # print(value_pos_neighbour)

          # t1 = value_pos_neighbour + lbp_corner
          # t2 = value_pos_corner + lbp_neigbour
          # t3 = value_neg_neighbour + lbp_corner
          # t4 = value_neg_corner + lbp_neigbour

          t1 = [value_pos_neighbour[k] + lbp_corner[k] for k in range(0,8)]
          t2 = [value_pos_corner[k] + lbp_neigbour[k] for k in range(0,8)]
          t3 = [value_neg_neighbour[k] + lbp_corner[k] for k in range(0,8)]
          t4 = [value_neg_corner[k] + lbp_neigbour[k] for k in range(0,8)]

          # print(t1)
          # print(t2)
          # print(t3)
          # print(t4)

          oc = [max(t1[k], t2[k], t3[k], t4[k]) for k in range(0,8)] 
          # print(oc)
          foc = [sigmoid(oc[k], slope, threshold) for k in range(0,8)] #---------------------------------------------------------------- #error here
          # print(foc)
          # print(sum(foc))
          orthoimg[i, j] = binary(foc)   #average - suggested by vaishali ma'am  # hopefully it should work #it should have been binary so did that later
          # focmatrix.append(foc)
          # print("next iter")
  return orthoimg

###FACE Feature Extraction

In [12]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qHd-N5hezjoJDzu_JkFQ8a6_xFht4Vq_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qHd-N5hezjoJDzu_JkFQ8a6_xFht4Vq_" -O train1.zip && rm -rf /tmp/cookies.txt
!unzip train1.zip

#https://drive.google.com/file/d/1qHd-N5hezjoJDzu_JkFQ8a6_xFht4Vq_/view?usp=sharing

subjects=os.listdir("/content/train")
for i in subjects:
    path1=os.path.join("/content/train",os.path.join(i,'wav'))
    path2=os.path.join("/content/train",os.path.join(i,'thr'))
    shutil.rmtree(path1,ignore_errors=True)
    shutil.rmtree(path2,ignore_errors=True)

Streaming output truncated to the last 5000 lines.
  inflating: train/sub_3/wav/3_1_2_1_519_1.wav  
  inflating: train/sub_3/wav/3_2_2_9_708_1.wav  
  inflating: train/sub_3/wav/3_1_2_9_711_1.wav  
  inflating: train/sub_3/wav/3_1_2_3_1148_1.wav  
  inflating: train/sub_3/wav/3_2_2_2_987_1.wav  
  inflating: train/sub_3/wav/3_1_2_4_1280_1.wav  
  inflating: train/sub_3/wav/3_1_2_4_100_1.wav  
  inflating: train/sub_3/wav/3_2_2_4_1150_1.wav  
  inflating: train/sub_3/wav/3_2_2_2_825_1.wav  
  inflating: train/sub_3/wav/3_1_2_4_1059_1.wav  
  inflating: train/sub_3/wav/3_2_2_7_672_1.wav  
  inflating: train/sub_3/wav/3_2_2_5_32_1.wav  
  inflating: train/sub_3/wav/3_2_2_2_459_1.wav  
  inflating: train/sub_3/wav/3_1_2_5_194_1.wav  
  inflating: train/sub_3/wav/3_2_2_5_1089_1.wav  
  inflating: train/sub_3/wav/3_1_2_5_1196_1.wav  
  inflating: train/sub_3/wav/3_2_2_9_757_1.wav  
  inflating: train/sub_3/wav/3_2_2_3_557_1.wav  
  inflating: train/sub_3/wav/3_2_2_6_1093_1.wav  
  inflating:

In [13]:
!mkdir facefeature_train
!mkdir facefeature_train/train

mkdir: cannot create directory ‘facefeature_train’: File exists
mkdir: cannot create directory ‘facefeature_train/train’: File exists


In [14]:
subjects=os.listdir("/content/train")
for i in subjects:
    path=os.path.join("/content/train",i)
    save_path=os.path.join("/content/facefeature_train",i)
    if not(os.path.isdir(save_path)):
        os.mkdir(save_path)
        os.mkdir(os.path.join(save_path,"rgb"))

    for j in os.listdir(os.path.join(path,'rgb')):
        print(os.path.join(path,j))
        img=cv2.imread(os.path.join(os.path.join(path,'rgb'),j),0)
        feature=ortho(img)
        cv2.imwrite(os.path.join(os.path.join(save_path,'rgb'),re.sub('.png','_feature.png',j)),feature)

/content/train/sub_20/20_1_2_8_434_1_7.png
/content/train/sub_20/20_2_2_3_386_1_7.png
/content/train/sub_20/20_1_2_5_828_1_7.png
/content/train/sub_20/20_1_2_3_180_1_7.png
/content/train/sub_20/20_2_2_5_725_1_7.png
/content/train/sub_20/20_1_2_9_1162_1_7.png
/content/train/sub_20/20_1_2_9_417_1_7.png
/content/train/sub_20/20_2_2_4_747_1_7.png
/content/train/sub_20/20_1_2_5_1021_1_7.png
/content/train/sub_20/20_2_2_5_646_1_7.png
/content/train/sub_20/20_1_2_7_1059_1_7.png
/content/train/sub_20/20_2_2_9_766_1_7.png
/content/train/sub_20/20_2_2_3_68_1_7.png
/content/train/sub_20/20_2_2_7_362_1_7.png
/content/train/sub_20/20_2_2_6_598_1_7.png
/content/train/sub_20/20_2_2_7_1038_1_7.png
/content/train/sub_20/20_2_2_6_189_1_7.png
/content/train/sub_20/20_2_2_6_174_1_7.png
/content/train/sub_20/20_2_2_3_1101_1_7.png
/content/train/sub_20/20_1_2_9_1035_1_7.png
/content/train/sub_20/20_1_2_7_1192_1_7.png
/content/train/sub_20/20_1_2_5_554_1_7.png
/content/train/sub_20/20_2_2_7_70_1_7.png
/conte

In [15]:
!zip -r facefeature_train.zip /content/facefeature_train

  adding: content/facefeature_train/ (stored 0%)
  adding: content/facefeature_train/sub_20/ (stored 0%)
  adding: content/facefeature_train/sub_20/rgb/ (stored 0%)
  adding: content/facefeature_train/sub_20/rgb/20_1_2_8_863_1_7_feature.png (deflated 2%)
  adding: content/facefeature_train/sub_20/rgb/20_2_2_1_832_1_7_feature.png (deflated 2%)
  adding: content/facefeature_train/sub_20/rgb/20_1_2_7_666_1_7_feature.png (deflated 2%)
  adding: content/facefeature_train/sub_20/rgb/20_2_2_5_646_1_7_feature.png (deflated 4%)
  adding: content/facefeature_train/sub_20/rgb/20_1_2_1_1_1_7_feature.png (deflated 1%)
  adding: content/facefeature_train/sub_20/rgb/20_1_2_9_366_1_7_feature.png (deflated 2%)
  adding: content/facefeature_train/sub_20/rgb/20_2_2_2_447_1_7_feature.png (deflated 2%)
  adding: content/facefeature_train/sub_20/rgb/20_1_2_3_736_1_7_feature.png (deflated 2%)
  adding: content/facefeature_train/sub_20/rgb/20_1_2_7_1059_1_7_feature.png (deflated 3%)
  adding: content/facefeat

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
!cp facefeature_train.zip '/content/gdrive/MyDrive/BM'
!ls -lt '/content/gdrive/MyDrive/BM' 